In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import datetime
import seaborn as sns
import keras

Using TensorFlow backend.


In [2]:
shops = pd.read_csv("data/shops.csv")
items = pd.read_csv("data/items.csv")
item_categories = pd.read_csv("data/item_categories.csv")
test = pd.read_csv("data/test.csv")
sales_train = pd.read_csv("data/sales_train.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [3]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
sales_train.shape

(2935849, 6)

In [6]:
# removing shop id and item id which are not in test
test_shop_ids = test['shop_id'].unique()
test_item_ids = test['item_id'].unique()
# Only shops that exist in test set.
sales_train = sales_train[sales_train['shop_id'].isin(test_shop_ids)]
# Only items that exist in test set.
sales_train = sales_train[sales_train['item_id'].isin(test_item_ids)]

In [7]:
sales_train.shape

(1224439, 6)

In [8]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')
#now we will create a pivot tabel by going so we get our data in desired form 
#we want get total count value of an item over the whole month for a shop 
# That why we made shop_id and item_id our indices and date_block_num our column 
# the value we want is item_cnt_day and used sum as aggregating function 
dataset = sales_train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [9]:
dataset.head()

item_cnt_day                            ...                    \
date_block_num            0  1  2  3  4  5  6  7  8  9  ... 24 25 26 27 28 29   
shop_id item_id                                         ...                     
2       30                 0  0  1  0  0  1  0  0  0  0 ...  0  0  0  0  0  0   
        31                 0  4  1  1  0  0  0  0  0  0 ...  0  0  0  0  0  0   
        32                 0  0  0  0  0  0  0  0  0  0 ...  0  0  1  0  0  0   
        33                 1  0  0  0  0  0  0  0  0  0 ...  0  1  0  1  1  0   
        53                 0  0  0  0  0  0  0  0  0  0 ...  0  1  0  0  0  0   

                             
date_block_num  30 31 32 33  
shop_id item_id              
2       30       0  0  0  0  
        31       0  0  0  1  
        32       0  1  0  0  
        33       1  0  1  0  
        53       0  0  0  0  

[5 rows x 34 columns]

In [10]:
# lets reset our indices, so that data should be in way we can easily manipulate
dataset.reset_index(inplace = True)

In [11]:
# lets check on our pivot table
dataset.head()

shop_id item_id item_cnt_day                      ...           \
date_block_num                            0  1  2  3  4  5  6  7 ... 24 25 26   
0                    2      30            0  0  1  0  0  1  0  0 ...  0  0  0   
1                    2      31            0  4  1  1  0  0  0  0 ...  0  0  0   
2                    2      32            0  0  0  0  0  0  0  0 ...  0  0  1   
3                    2      33            1  0  0  0  0  0  0  0 ...  0  1  0   
4                    2      53            0  0  0  0  0  0  0  0 ...  0  1  0   

                                     
date_block_num 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  
1               0  0  0  0  0  0  1  
2               0  0  0  0  1  0  0  
3               1  1  0  1  0  1  0  
4               0  0  0  0  0  0  0  

[5 rows x 36 columns]

In [12]:
# Now we will merge our pivot table with the test_data because we want to keep the data of items we have
# predict
dataset = pd.merge(test,dataset,on = ['item_id','shop_id'],how = 'left')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning:

merging between different levels can give an unintended result (1 levels on the left, 2 on the right)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [13]:
# lets fill all NaN values with 0
# dataset.fillna(0,inplace = True)
dataset.fillna(0, inplace=True)
# lets check our data now 
dataset.head()

,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# we will drop shop_id and item_id because we do not need them
# we are teaching our model how to generate the next sequence 
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
temp = dataset.clip(0, )

In [16]:
temp.max()

(item_cnt_day, 0)      169.0
(item_cnt_day, 1)      117.0
(item_cnt_day, 2)      259.0
(item_cnt_day, 3)      151.0
(item_cnt_day, 4)      504.0
(item_cnt_day, 5)      766.0
(item_cnt_day, 6)      799.0
(item_cnt_day, 7)      820.0
(item_cnt_day, 8)      950.0
(item_cnt_day, 9)      978.0
(item_cnt_day, 10)     989.0
(item_cnt_day, 11)    1305.0
(item_cnt_day, 12)     899.0
(item_cnt_day, 13)     941.0
(item_cnt_day, 14)     776.0
(item_cnt_day, 15)     597.0
(item_cnt_day, 16)     602.0
(item_cnt_day, 17)     771.0
(item_cnt_day, 18)     563.0
(item_cnt_day, 19)     591.0
(item_cnt_day, 20)     639.0
(item_cnt_day, 21)     634.0
(item_cnt_day, 22)     772.0
(item_cnt_day, 23)    1209.0
(item_cnt_day, 24)    1000.0
(item_cnt_day, 25)     257.0
(item_cnt_day, 26)     174.0
(item_cnt_day, 27)     813.0
(item_cnt_day, 28)     742.0
(item_cnt_day, 29)     444.0
(item_cnt_day, 30)     482.0
(item_cnt_day, 31)     436.0
(item_cnt_day, 32)     473.0
(item_cnt_day, 33)    2253.0
dtype: float64

In [17]:
# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

(214200, 33, 1) (214200, 1) (214200, 33, 1)


In [18]:
X_test.shape

(214200, 33, 1)

In [19]:
# importing libraries required for our model
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.optimizers import Adam

In [20]:
epochs = 100
# our defining our model 
my_model = Sequential()
my_model.add(LSTM(units = 64, activation='tanh', input_shape = (33,1), return_sequences=True))
my_model.add(Dropout(0.5))
my_model.add(LSTM(units= 32, activation='tanh'))
my_model.add(Dropout(0.5))
my_model.add(Dense(1))
# opt = Adam(lr=1e-3, decay=1e-3/epochs)
my_model.compile(loss = 'mse',optimizer = 'rmsprop', metrics = ['mean_squared_error'])
my_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 33, 64)            16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 33, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist = my_model.fit(X_train,y_train,validation_split=0.2, batch_size = 4096,epochs = epochs)

In [ ]:
plt.plot(hist.history['loss'])
plt.title('train loss')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.savefig('train_loss.png')

In [ ]:
plt.plot(ypre.history['loss'])
plt.title('train loss')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.show()

In [ ]:
# creating submission file 
submission_pfs = my_model.predict(X_test)
# we will keep every value between 0 and 20
submission_pfs = submission_pfs.clip(0,20)
# creating dataframe with required columns 
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe
submission.to_csv('sub_pfs.csv',index = False)

In [24]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= 'train_loss.png')